<a href="https://colab.research.google.com/github/HaripriyaVennam/Neural-Networks-and-Deep-Learning/blob/main/exam/Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin


In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data to be in the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define number of classes
num_classes = 10


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [3]:
from tensorflow.keras.layers import Input

# Function to create the CNN model
def create_cnn_model(optimizer='adam', learning_rate=0.001):
    model = Sequential([
        Input(shape=(32, 32, 3)),  # Specify input shape using the Input layer
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [4]:
class KerasModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, optimizer='adam', learning_rate=0.001):
        self.optimizer = optimizer
        self.learning_rate = learning_rate

    def fit(self, X, y):
        self.model = create_cnn_model(self.optimizer, self.learning_rate)
        self.model.fit(X, y, epochs=10, batch_size=64, verbose=0, validation_split=0.1)
        return self

    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=-1)

    def score(self, X, y):
        return self.model.evaluate(X, y, verbose=0)[1]  # Return accuracy

# Wrapper for GridSearchCV
model = KerasModelWrapper()


In [5]:
param_grid = {
    'optimizer': [Adam, SGD],
    'learning_rate': [0.001, 0.01],
}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
param_grid = {
    'optimizer': [Adam],  # Start with just Adam optimizer for faster testing
    'learning_rate': [0.001, 0.01],  # Smaller set of learning rates
}

# Use RandomizedSearchCV for faster search (less exhaustive)
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                   n_iter=2, cv=2, n_jobs=-1, verbose=1, random_state=42)
random_search_result = random_search.fit(x_train, y_train)

# Best Parameters from Randomized Search
print(f"Best Parameters: {random_search_result.best_params_}")
print(f"Best Validation Accuracy: {random_search_result.best_score_}")

# Get the best model
best_model = random_search_result.best_estimator_


Fitting 2 folds for each of 2 candidates, totalling 4 fits


In [ ]:
# Callback for ReduceLROnPlateau to reduce learning rate based on validation loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Fit the best model with the callback
history = best_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[reduce_lr])


In [ ]:
# Create subplots for Loss and Accuracy
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot Training and Validation Loss
axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()

# Plot Training and Validation Accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_title('Training and Validation Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(10), yticklabels=np.arange(10))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
# Compute Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_classes)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()


In [ ]:
# Compute ROC curve and AUC for one-vs-rest classification
fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])  # Using binary classification for simplicity (one-vs-rest)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()


Explanation of Key Steps Leading to Improved Model Performance:
Callback (ReduceLROnPlateau): By automatically reducing the learning rate when validation loss plateaus, the model is able to make finer adjustments and avoid overshooting the optimal solution.

Grid Search for Hyperparameter Optimization: By tuning the optimizer (Adam vs. SGD) and learning rate, we ensure that the model trains with the optimal settings, helping to achieve better accuracy and reduced validation loss.

Model Architecture: A convolutional neural network (CNN) was chosen to handle image data, where convolution layers extract important features from the images. Dropout was added to prevent overfitting.

Confusion Matrix: This visualization provided a clear understanding of which classes the model was misclassifying, helping to identify areas for improvement in future models.

Precision-Recall and ROC Curves: These visualizations provide more granular performance metrics, especially useful for understanding the model’s behavior for each class, beyond accuracy.